# Introducción

## PROBLEMA PROPUESTO

Resolver un laberinto arbitrario en donde un agente deberá encontrar un camino desde un punto de inicio hasta una salida, inicialmente suponiendo que solo hay un camino a encontrar.  
El laberinto será representado como una matriz de $n$ por $m$, sus elementos podrán ser solamente $0$ y $1$, siendo $M$ el laberinto de prueba:
$$
M =
\begin{bmatrix}
0 & 1 & 0 & 0 & 0\\  
0 & 1 & 0 & 1 & 0\\
0 & 0 & 0 & 1 & 0\\
1 & 1 & 0 & 1 & 0\\  
0 & 0 & 0 & 0 & 0\\
\end{bmatrix}
$$


### Estado
El estado $s$ representará la posición actual del agente en el laberinto tal que es representado por el vector
$$
\mathbf{s} =
\begin{bmatrix}
x \\
y
\end{bmatrix}
$$

### Operación sucesor

Desde un estado $(x, y)$, el agente puede moverse cuando mucho en cuatro direcciones, las direcciones en las que se puede mover dependen de si las celdas vecinas son paredes, caminos o bordes del mundo. Los cuatro movimientos disponibles serán:

| Movimiento | Nueva posición        |
|------------|-----------------------|
| Arriba     | $(\text{fila}-1, \text{col})$   |
| Abajo      | $(\text{fila}+1, \text{col})$   |
| Izquierda  | $(\text{fila}, \text{col}-1)$   |
| Derecha    | $(\text{fila}, \text{col}+1)$   |

### CONDICIÓN DE META
La condicion de meta se cumple cuando el estado actual del agente (la posición $\mathbf{p}_{\text{actual}}$) es la posicion de salida $\mathbf{p}_{\text{meta}}$ del laberinto. Por ejemplo, si la meta es la coordenada $\mathbf{p}_{\text{meta}} = (x, y)$, el algoritmo termina cuando $\mathbf{p}_{\text{actual}} = \mathbf{p}_{\text{meta}}$.



### HEURISTICA PARA A*
Heuristica apropiada es la **distancia Manhattan** ya que los movimientos están restringidos a 4 direcciones (diagonales no son posibles):
$$ h(x_i,y_i) = | x - x_i| + |y - y_i|$$
Siendo que:
- $x$ y $y$ son las posiciones meta.
- $x_i$ y $y_i$ son las posiciones de la iteración $i$.

#### Justificacion
La distancia de Manhattan es una heuristica admisible para este problema por que:
- Admisibilidad: Nunca sobreestima el costo real de llegar a la meta. En un laberinto de cuadrícula, el camino más corto siempre será igual o mayor que la suma de las distancias horizontales y verticales.
- Buen estimado: la distancia Manhattan da un mejor estimado que trazar directamente una recta que indique su distancia real con la meta, esto porque los movimientos diagonales no son posibles aquí.

## Desarrolo

### Preparación de las búsquedas

Definición de las funciones que definen el comportamiento de la búsqueda:

In [2]:
from src.SimpleSearch import BlindSearch, node

# Definir el laberinto y los estados
maze = [
    [0, 1, 0, 0, 0],
    [0, 1, 0, 1, 0],
    [0, 0, 0, 1, 0],
    [1, 1, 0, 1, 0],
    [0, 0, 0, 0, 0]
]

#ESTADOS
initial_state = (0, 0)
goal_state = (4, 4)

In [ ]:


# Función de sucesores para BlindSearch
def succ(nodo):
    state = nodo.state
    row, col = state
    moves = []

    #OPERACION SUCESOR
    # Estoy seguro que hay una forma más idiomática con list comprehensions
    for action, (dr, dc) in {
        'up': (-1, 0),
        'down': (1, 0),
        'left': (0, -1),
        'right': (0, 1)
    }.items():
        new_row, new_col = row + dr, col + dc  # CORREGIDO: era row_i, row_j
        if 0 <= new_row < len(maze) and 0 <= new_col < len(maze[0]):
            if maze[new_row][new_col] == 0:
                moves.append(node((new_row, new_col), parent=nodo, depth=nodo.depth+1, op=action))
    return moves

#CONDICION DE META
def maze_is_solved(nodo, goal_state):
    print("Comparando:", nodo.state, goal_state)
    return nodo.state == goal_state

#HEURISTICA
def heuristica_manhattan(nodo, goal_state):
    row, col = nodo.state
    goal_row, goal_col = goal_state
    return abs(row - goal_row) + abs(col - goal_col)

# Nodo inicial
nodo_inicial = node(initial_state)


Agregamos la definición de el problema de prueba:

Antes de empezar con las búsquedas agregamos una función para dibujar los laberintos:

In [4]:
# Función para imprimir el laberinto con el camino los "*" muestran el camino a la solucion
def print_maze_with_path(maze, path, start, goal):
    maze_copy = [row.copy() for row in maze]
    row, col = start
    for move in path:
        if move == 'up': row -= 1
        if move == 'down': row += 1
        if move == 'left': col -= 1
        if move == 'right': col += 1
        maze_copy[row][col] = '*'
    s_row, s_col = start
    g_row, g_col = goal
    maze_copy[s_row][s_col] = 'S'
    maze_copy[g_row][g_col] = 'G'
    for r in maze_copy:
        print(' '.join(str(c) for c in r))
    print()


### Búsquedas

In [ ]:
print("=== INICIANDO BFS ===")
print("Estado inicial:", initial_state)
print("Estado meta:", goal_state)

busqueda_bfs = BlindSearch(
    start=node(initial_state),  
    successor=succ,
    goal=maze_is_solved,
    strategy="bfs",
    goal_state=goal_state
)

print("\n--- Ejecutando BFS ---")
sol_bfs = busqueda_bfs.find(debug=True)
print("--- BFS Terminado ---\n")

if sol_bfs.state != "Solution not found":
    camino_bfs = [step[1] for step in sol_bfs.getPath()][1:]
    print("BFS - Camino encontrado:")
    print_maze_with_path(maze, camino_bfs, initial_state, goal_state)
else:
    print("BFS: No se encontró solución")

Comparando: (0, 0) (4, 4)
Comparando: (0, 0) (4, 4)
Comparando: (1, 0) (4, 4)
finished in 2 iterations
Se acabaron los estados: 0
Comparando: (1, 0) (4, 4)
BFS 
S 1 0 0 0
0 1 0 1 0
0 0 0 1 0
1 1 0 1 0
0 0 0 0 G



# Depth First Search

In [ ]:
# DFS
busqueda_dfs = BlindSearch(
    start=node(initial_state),
    successor=succ,
    goal=maze_is_solved,
    strategy="dfs",
    goal_state=goal_state
)
sol_dfs = busqueda_dfs.find()
camino_dfs = [step[1] for step in sol_dfs.getPath()][1:]

print("DFS")
print_maze_with_path(maze, camino_dfs, initial_state, goal_state)


# A\* search

In [ ]:
busqueda_astar = BlindSearch(
    start=node(initial_state),
    successor=succ,
    goal=maze_is_solved,
    strategy="a*",
    goal_state=goal_state,
    heuristic=heuristica_manhattan
)
sol_astar = busqueda_astar.find()
camino_astar = [step[1] for step in sol_astar.getPath()][1:]


print("A*")
print_maze_with_path(maze, camino_astar, initial_state, goal_state)

# Medición de tiempo

import time

def run_and_measure(search_obj):
    start_time = time.time()
    solution = search_obj.find()
    elapsed = time.time() - start_time
    path = [step[1] for step in solution.getPath()][1:]
    nodos_expandidos = getattr(search_obj, 'expanded', 'No disponible')
    return elapsed, len(path), nodos_expandidos

# calcular BFS
bfs_time, bfs_length, bfs_expanded = run_and_measure(busqueda_bfs)
# calcular DFS
dfs_time, dfs_length, dfs_expanded = run_and_measure(busqueda_dfs)
# calcular A*
astar_time, astar_length, astar_expanded = run_and_measure(busqueda_astar)

print("Comparación de algoritmos:")
print(f"BFS:  Tiempo={bfs_time:.4f}s, Longitud camino={bfs_length}, Nodos expandidos={bfs_expanded}")
print(f"DFS:  Tiempo={dfs_time:.4f}s, Longitud camino={dfs_length}, Nodos expandidos={dfs_expanded}")
print(f"A*:   Tiempo={astar_time:.4f}s, Longitud camino={astar_length}, Nodos expandidos={astar_expanded}")